In [1]:
import re
from selenium import webdriver

import time

from bs4 import BeautifulSoup
import requests

import pandas as pd

# Scraping the Oyez Project Website

## Selenium + BeautifulSoup

In [2]:
a = 1

driver = webdriver.Chrome()
oyez_list = []

while a < 73:
    oyez_case_dict = {}
    
    driver.get("https://www.oyez.org/cases/2016")
    
    time.sleep(3)
    granted = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/ng-include/ul/li['+ str(a) +']/aside/div[1]')
    oyez_case_dict['granted'] = granted.text.replace('GRANTED', '').strip()
    
    argued = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/ng-include/ul/li['+ str(a) +']/aside/div[2]')
    oyez_case_dict['argued'] = argued.text.replace('ARGUED', '').strip()
    
    decided = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/ng-include/ul/li['+ str(a) +']/aside/div[3]')
    oyez_case_dict['decided'] = decided.text.replace('DECIDED', '').strip()
    
    summary = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/ng-include/ul/li['+ str(a) +']/div')
    oyez_case_dict['summary'] = summary.text.strip()
    
    title = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/ng-include/ul/li['+ str(a) +']/h2/a')
    title.click()
    
    time.sleep(3)
    
    doc = BeautifulSoup(driver.page_source, 'html.parser')
    cells = doc.find_all('div', attrs = {'class': 'row'})
    
    location = cells[0].find('div', attrs = {'class': 'cell ng-scope'})
    if location:
        location = location.a['iframe-url']
        oyez_case_dict['location'] = re.findall(r'q=(.*)', location)[0]
    
    time.sleep(1)
    petitioner = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/aside/div[1]/div[1]')
    oyez_case_dict['petitioner'] = petitioner.text.replace('PETITIONER', '').strip()
    
    respondent = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/aside/div[1]/div[2]')
    oyez_case_dict['respondent'] = respondent.text.replace('RESPONDENT', '').strip()
    
    docket = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/aside/div[2]/div[1]')
    oyez_case_dict['docket'] = docket.text.replace('DOCKET NO.', '').strip()
    
    
    time.sleep(1)
    fact = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/section[1]/div')
    oyez_case_dict['descriptions'] = fact.text.strip()

    time.sleep(1)
    conclusion = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/section[1]/div')
    oyez_case_dict['conclusion'] = conclusion.text.strip()

    oyez_list.append(oyez_case_dict)
    
    time.sleep(1)

driver.close()

WebDriverException: Message: chrome not reachable
  (Session info: chrome=59.0.3071.115)
  (Driver info: chromedriver=2.30.477690 (c53f4ad87510ee97b5c3425a14c0e79780cdf262),platform=Mac OS X 10.12.5 x86_64)


In [ ]:
b = 1

oyez_list2 = []
driver = webdriver.Chrome()

while b < 73:

    oyez_case_dic = {}

    driver.get("https://www.oyez.org/cases/2016")
    
    time.sleep(3)

    title = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/ng-include/ul/li['+ str(b) +']/h2/a')
    title.click()

    time.sleep(3)

    doc = BeautifulSoup(driver.page_source, 'html.parser')
        
    maj_opin = doc.find('li', attrs = {'ng-if':'document.justia_opinion_url'})
    if maj_opin:
        oyez_case_dic['docket'] = maj_opin.a['href']

    time.sleep(1)

    docket = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/aside/div[2]/div[1]')
    oyez_case_dic['docket'] = docket.text.replace('DOCKET NO.', '').strip()


    time.sleep(1)
    
    try:
        vote = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/section[3]/div[1]/div[3]/ul/li/figure/figcaption/h3')
        oyez_case_dic['vote'] = vote.text.strip()
    except:
        pass

    try:
        s_conclusion = driver.find_element_by_xpath('/html/body/div/div/div[3]/main/article/div/section[3]/div[1]/div[3]/ul/li/figure/figcaption/p')
        oyez_case_dic['s_conclusion'] = s_conclusion.text.strip()
    except:
        pass
    
    oyez_list2.append(oyez_case_dic)
    
    time.sleep(1)
    
driver.close()

In [ ]:
info_df2 = pd.DataFrame(oyez_list2)
info_df2

In [ ]:
info_df = pd.DataFrame(oyez_list)
info_df

# Scraping the Supreme Court Website

In [ ]:
driver = webdriver.Chrome()
driver.get("https://www.supremecourt.gov/oral_arguments/argument_transcript.aspx")
doc = BeautifulSoup(driver.page_source, 'html.parser')
caseno_list = doc.find_all('a')
case_list = []
case_dict = {}

caseno_list = caseno_list[86:150]

caseno_flist = []
caseno_link = []
file_name_list = []
for case in caseno_list:
    tran_link = case['href']
    caseno_link.append(tran_link)
    regex10 = r'[/]([0-9]{2}-[0-9]+_[0-9a-z]+).pdf$'
    file_name = re.findall(regex10, tran_link)
    for file in file_name:
        file_name_list.append(file)
    case = case.text.strip().replace('.','')
    caseno_flist.append(case)
    
name_list = doc.find_all('span')
name_list = name_list[18:82]
name_flist = []
for name in name_list:
    name = name.text.strip()
    name_flist.append(name)
    
    
# date_list = doc.find_all('tr')
# date_list = date_list[3:73]
# date_flist = []
# for date_cell in date_list:
#     date = date_cell.find_all('td')[1].text.strip()
#     if date != 'Date Argued':
#         date_flist.append(date)
        
driver.close()

df = pd.DataFrame({
    'docket': caseno_flist,
    'caseName': name_flist,
    #'date argued': date_flist,
    'transcriptLink': caseno_link,
    'fileName': file_name_list
})

In [ ]:
df

In [ ]:
f_df = df.merge(info_df, left_on="docket", right_on="docket")
f_df

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

%matplotlib inline

# Scraping the oral arugment transcripts

## regular expressions

In [ ]:
def scraping(sample_transcript):
    trans_list = []
    
    
    my_reg = r"Alderson Reporting Company[^S]*Subject to Final Review"
    sample_transcript = re.sub(my_reg," ",sample_transcript)
    
    #sample_transcript = sample_transcript.replace('Alderson Reporting Company', '')
    #sample_transcript = sample_transcript.replace('Official - Subject to Final Review', '')

    regex = r"\b[12]{0,1}[0-9]?\b"
    sample_transcript = re.sub(regex, '', sample_transcript)

    chop_top = sample_transcript.split("PROCEEDINGS")
    sample_transcript = chop_top[1]

    regex2 = r"(submitted.[)])"
    chop_end = re.split(regex2,sample_transcript)
    sample_transcript = chop_end[0] + chop_end[1]

    regex3 = r'([A-Z]+[.]{0,1}[ ]{0,1}[A-Z]+\W{0,2}[A-Z]+):'

    transcript_list = re.split(regex3,sample_transcript)
    transcript_list

    transcript_list = transcript_list[1:]

    d = 0
    speaker_list = []
    word_list = []
    
    
    
    for speaker in transcript_list:
        if d % 2 == 0:
            speaker_list.append(speaker.strip())
        else:
            word_list.append(speaker.strip())
        d = d + 1
    
    for item1,item2 in zip(speaker_list, word_list):
        trans_dict = {}
        trans_dict["speaker"] = item1
        trans_dict["word"] = item2
        trans_list.append(trans_dict)
        
    return trans_list

In [ ]:
all_trans_list = []
z = 0

for file, case in zip(file_name_list, caseno_flist):
    f = open('/Users/Shiying/Documents/ColumbiaLede/data_database/supreme_court/pdfs/' + file + '.txt', 'rb')
    
    try:
        #print('/Users/Shiying/Documents/ColumbiaLede/data_database/supreme_court/pdfs/' + file + '.txt')
        this_transcript = f.read().decode('utf8', 'ignore')
        trans_list = scraping(this_transcript)
        for item in trans_list:
            item['docket'] = case
        
        all_trans_list.extend(trans_list)
        
    except:
        pass
    
df2 = pd.DataFrame(all_trans_list)

In [ ]:
df2

In [ ]:
df3 = df2[df2['speaker'].str.contains('JUSTICE')]

In [ ]:
speak_lines = df3.sort_values(by='speaker')

In [ ]:
lines_out = speak_lines.groupby(['docket','speaker'])['word'].apply(lambda x: "<p>%s</p>" % '</p><p> '.join(x)).reset_index(name='lines')
lines_out

In [ ]:
lines_out['line_dict'] = lines_out.apply(lambda row: {row['speaker']:row['lines']}, axis=1)
lines_out.to_csv('line_out.csv', index= False)

In [ ]:
lines_out

In [ ]:
result_list = []
for item in lines_out['docket'].unique():
    result = {}
    for x in lines_out[lines_out['docket'] == item]['line_dict']:
        result.update(x)
    result_list.append(result)

In [ ]:
len(result_list)

In [ ]:
lines_out.apply(lambda x: "%s" % '</p><p> '.join(x)).reset_index(name='lines')

In [ ]:
df4 = df3.groupby('docket')['speaker'].value_counts().reset_index(name = 'per_docket')

In [ ]:
df4['string'] = df4['speaker'].str.title() + ": " + df4['per_docket'].map(str)
df4

In [ ]:
output = df4.groupby('docket')['string'].apply(lambda x: ",  ".join(x)).reset_index(name='article')
output2 = df4.groupby('docket')['string'].apply(lambda x: "<p>%s</p>" % '</p><p> '.join(x)).reset_index(name='article')
output

In [ ]:
output['f_speaker_line'] = result_list
output

In [ ]:
df3['speaker'].value_counts()

In [ ]:
justices = df3.groupby('speaker')['docket'].value_counts().reset_index(name='times')
justices

In [ ]:
justices2 = output2
justices2

In [ ]:
justices2['speak_in'] = '0'
justices2['color'] = 'CE21C3'
justices2 = justices2[['docket', 'speak_in', 'color']]

In [ ]:
justices2

In [ ]:
justices.loc[justices.speaker == 'CHIEF JUSTICE ROBERTS', 'speak_in'] = '1'
justices.loc[justices.speaker == 'JUSTICE ALITO', 'speak_in'] = '2'
justices.loc[justices.speaker == 'JUSTICE BREYER', 'speak_in'] = '3'
justices.loc[justices.speaker == 'JUSTICE GINSBURG', 'speak_in'] = '4'
justices.loc[justices.speaker == 'JUSTICE GORSUCH', 'speak_in'] = '5'
justices.loc[justices.speaker == 'JUSTICE KAGAN', 'speak_in'] = '6'
justices.loc[justices.speaker == 'JUSTICE KENNEDY', 'speak_in'] = '7'
justices.loc[justices.speaker == 'JUSTICE SOTOMAYOR', 'speak_in'] = '8'

In [ ]:
justices

In [ ]:
speak_perdocket = justices.groupby('docket')['speak_in'].value_counts().reset_index(name='times')
speak_perdocket

In [ ]:
speak_color = justices.groupby('docket')['color'].value_counts().reset_index(name='times')
speak_color

In [ ]:
f_speak_perdocket = pd.concat([speak_perdocket, justices2])
f_speak_perdocket

In [ ]:
f_speak_color = pd.concat([speak_color, justices2])
f_speak_color.groupby('docket')['color'].value_counts()

In [ ]:
a = f_speak_perdocket.groupby('docket')['speak_in'].apply(lambda x: ','.join(x).split(',')).reset_index(name='speak_list')
a

In [ ]:
a['color'] = "['CE21C3', '0BB2E8','37CB00','8A04D8','D82C86','0BD488','000000','7E3B0D']"

In [ ]:
a

# Merge all the tables

In [ ]:
f_df = f_df.merge(output, left_on = 'docket', right_on = 'docket')
f_df = f_df.merge(a, left_on = 'docket', right_on = 'docket')
f_df = f_df.merge(info_df2, left_on = 'docket', right_on = 'docket')
f_df

# Turn the longtitude and latitude into geopandas points

In [ ]:
f_df['lat'] = f_df['location'].apply(lambda x: x.split(',')[0])
f_df['lon'] = f_df['location'].apply(lambda x: x.split(',')[1])
f_df['lat'] = f_df['lat'].astype(float)
f_df['lon'] = f_df['lon'].astype(float)

In [ ]:
ff_df = f_df[['caseName', 'docket', 'granted', 'argued', 'decided', 'lat', 'lon', 'summary', 'vote', 's_conclusion','f_speaker_line', 'descriptions', 'conclusion', 'article', 'speak_list', 'color']]
ff_df

In [ ]:
gf_df = gpd.GeoDataFrame(ff_df)

In [ ]:
gf_df

In [ ]:
gf_df['geometry'] = gf_df.apply(lambda row: Point(row.lon, row.lat), axis=1)
gf_df['speak_list'] = gf_df.speak_list.astype(str)

In [ ]:
gf_df.to_file('case500.json', driver='GeoJSON')